# Coursera Capstone Project
# Name: Shreyas Iyengar
# Occupation: Student

## This Notebook will be used for all of the Capstone Assignments, and will use markdown to organize the sections based on each week, and then subsections for each week.







## Week 1: Capstone Project Notebook
### This week, we have to create a notebook, with our name, and the title "Cousera Capstone Project". This will also include importing pandas and numpy, and then finally printing "Hello Capstone Project Course". 

### For ease of use, I have included the import statements at the top of the notebook as I will be using the import statements in other sections of the notebook. ###


In [ ]:
print("Hello Capstone Project Course")

In [1]:
import pandas as pd 
import numpy as np 

from bs4 import BeautifulSoup
import requests 

import geocoder 
import folium
from geopy.geocoders import Nominatim

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans




## Week 3: Segmenting and Clustering Neighborhoods in Toronto
### Following the lab where we had to work with the New York Neighborhoods and create an clustering analysis based on the venues in each neighborhood, we have to replicate the same idea using the city of Toronto, with some added stuff, including some webscraping to grab the data and then further data wrangling and cleaning to suit our purposes. For this assignment, I followed the analysis that we tried to do with the New York data, which was clustering based on most common venues in the area. That is replicated below.

#### Here we are trying to grab the data from the wikipedia page of postal codes in Canada, mader easier by the fact that postal codes in Toronto start with the letter M.

In [ ]:
#grab URL and store it in a URL variable
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#obtain raw html
r = requests.get(URL)
#parse into beautifulsoup 
soup = BeautifulSoup(r.content) 
#find required content
table = soup.findAll('table',attrs ={'rules':'all','cellspacing':0,}) 
#convert to text
a = table[0].text

"""
beautiful soup text has a lot of newlines. split by new line, and then run through
the rest of the code to remove the newline and replace it with an empty char
"""
b = a.split("\n")
for word in b:
    word.replace("\n", "")
    word.replace("\n", "")
    word.replace("\n", "")
#create a list
c = []
#now only add stuff that isn't an empty char or does not contain Not Assigned
for word in b:
    if "Not assigned" not in word and word != "":
        c.append(word)
#create a new table to store results
table_contents=[]

for word in c:
    #dict to store values for each neighborhood
    cell = {}
    #Since each postal code is 3 digits, go till the third digit and set it as the postal code value
    cell['Postal Code'] = word[:3]
    #set a temp variable equal to the rest of the word, which has the borough and the neighborhood
    temp = word.replace(word[:3], '')
    #split by the parenthesis, and grab the first half which is the borough, set it to the borough of the cell
    cell['Borough'] = temp.split('(')[0]
    #same logic for the neighborhood, just strip all the uneeded characters
    cell['Neighborhood'] = (((((temp).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
    #append the dict to the table
    table_contents.append(cell)
#convert to df
df=pd.DataFrame(table_contents)
#handle special cases
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

    
    
    

In [ ]:
#print the shape 
df.shape

In [ ]:

df.head()

#### Couldn't get the geocoder package to work, so I used the Geospatial dataset to grab the Latitude and Longitude for each neighborhood


In [ ]:
f = open("/Users/shreyasiyengar/VSC/Coursera_Capstone/Geospatial_Coordinates.csv", "r")
df2 = pd.read_csv("/Users/shreyasiyengar/VSC/Coursera_Capstone/Geospatial_Coordinates.csv",",",header = 0)
df2.head()




In [ ]:
df.head()

#### Here I merged df, which had the postal code borough and neighborhood with df2 which contained the Lat and Long coordinates of each postal code.

In [ ]:
result = pd.merge(df, df2[['Postal Code', 'Latitude', 'Longitude']], )
result

#### Here we're grabbing the coordinates of the city of Toronto to generate the base map upon which we will overlay the neighborhoods and clusters.

In [ ]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="trawno_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))


#### Below we are printing the base map with each neighborhood marked in red and white

In [ ]:
map_trawno = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#ffffff',
        fill_opacity=0.7,
        parse_html=False).add_to(map_trawno)  
    
map_trawno

#### This cell is for prepping for using the Foursquare places API

In [ ]:
CLIENT_ID = 'KOGTV4EAQCBU054ZKLCRMAINCATFAHUJWLHPTAUZ4A3N33SG' # your Foursquare ID
CLIENT_SECRET = '4UAMZ5NN21XIQHRK0MRSJ20TYYMF5MABJLQAQEDLRLEPTWGA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
#### The following code, following the New York Lab, gets the nearby venues for a neighborhood using the lat-long, we then use this function to grab the nearby venues for each neighborhood in our dataset.

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### All the nearby venues for each neighboorhood are stored in trawno_venues, and the name of each neighborhood is pritned out.

In [ ]:
trawno_venues = getNearbyVenues(names=result['Neighborhood'],
                                   latitudes= result['Latitude'],
                                   longitudes=result['Longitude']
                                  )

In [ ]:
trawno_venues.head()

In [ ]:
print('There are {} unique categories.'.format(len(trawno_venues['Venue Category'].unique())))

#### One Hot encoding for later grouping by mean.

In [ ]:
# one hot encoding
trawno_onehot = pd.get_dummies(trawno_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
trawno_onehot['Neighborhood'] = trawno_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [trawno_onehot.columns[-1]] + list(trawno_onehot.columns[:-1])
trawno_onehot = trawno_onehot[fixed_columns]

trawno_onehot.head()

In [ ]:
trawno_grouped = trawno_onehot.groupby('Neighborhood').mean().reset_index()
trawno_grouped

#### the code below grabs the top 5 venues for each neighborhood and prints the frequency of each venue 

In [ ]:
num_top_venues = 5

for hood in trawno_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = trawno_grouped[trawno_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

#### Grab the most common venues, limit by 10.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = trawno_grouped['Neighborhood']

for ind in np.arange(trawno_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(trawno_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

#### Clustering Analysis

In [ ]:
# set number of clusters
kclusters = 5

trawno_grouped_clustering = trawno_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(trawno_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

trawno_merged = result

# merge trawno_grouped with trawno_data to add latitude/longitude for each neighborhood
trawno_merged = trawno_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

trawno_merged.head() # check the last columns!

In [ ]:
#For some reason the Cluster Labels are converted to float which won't work with the mapping,so convert to ints.
trawno_merged['Cluster Labels'] = trawno_merged['Cluster Labels'].fillna(0.0).astype(int)
trawno_merged.head()









#### Map out each Cluster.

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(trawno_merged['Latitude'], trawno_merged['Longitude'], trawno_merged['Neighborhood'], trawno_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
trawno_merged.loc[trawno_merged['Cluster Labels'] == 0, trawno_merged.columns[[1] + list(range(5, trawno_merged.shape[1]))]]




In [ ]:
trawno_merged.loc[trawno_merged['Cluster Labels'] == 1, trawno_merged.columns[[1] + list(range(5, trawno_merged.shape[1]))]]


In [ ]:
trawno_merged.loc[trawno_merged['Cluster Labels'] == 2, trawno_merged.columns[[1] + list(range(5, trawno_merged.shape[1]))]]


In [ ]:
trawno_merged.loc[trawno_merged['Cluster Labels'] == 3, trawno_merged.columns[[1] + list(range(5, trawno_merged.shape[1]))]]


In [ ]:
trawno_merged.loc[trawno_merged['Cluster Labels'] == 4, trawno_merged.columns[[1] + list(range(5, trawno_merged.shape[1]))]]


## Cluster Data Analysis

Cluster 0: Most likely very dense, very people focused areas, as there are a lot of food joints, restaurants and cafes in the area, and not a lot of parks in many neighborhoods, implying that there isn't a lot of space for parks.

Cluster 1: Maybe a lot of immigrants, considering that there are a lot of mexican and middle eastern restaurants in the area. Might also not be as dense as cluster 0, considering there are a lot more parks than food destinations in the area.

Cluster 2: Seems further away from city center, as there are more parks and trails, and some of these neighborhoods might be closer to the airports seeing as there is an airport as a venue in one of the boroughs.

Cluster 3: Maybe more suburban as it has more variety in venues, as well as more pools, baseball fields, and a variety of services and restaurants

Cluster 4: Not really sure about the distinguishing feature in this cluster, but I would assume that this is in a more dense area, becasue of the restuaruants and the gyms in the area. 

##

## Week 4 and 5 Capstone Project - The Battle of Neighborhoods

### For this project, I want to find out which City, New York City or Toronto, is more vegan friendly, so that I can build a vegan restaruant in the city that is more receptive towards vegan restaurants, and would therefore want a vegan restaurant more. 


## Firstly, we need to make a table for New York and Toronto, which contains the Neighborhoods of each city, the postal codes/zip codes, borough, and the Lat-Long coordinates for each neighborhood.


In [2]:
#New York

# #grab URL and store it in a URL variable
NURL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json"

print(NURL)

# #obtain raw html
nyr = requests.get(NURL).json()

nydata = nyr["features"]

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
nyneigh = pd.DataFrame(columns=column_names)    

for data in nydata:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    nyneigh = nyneigh.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

nyneigh.head()


https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [8]:
#toronto

#grab URL and store it in a URL variable
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#obtain raw html
r = requests.get(URL)
#parse into beautifulsoup 
soup = BeautifulSoup(r.content) 
#find required content
table = soup.findAll('table',attrs ={'rules':'all','cellspacing':0,}) 
#convert to text
a = table[0].text

"""
beautiful soup text has a lot of newlines. split by new line, and then run through
the rest of the code to remove the newline and replace it with an empty char
"""
b = a.split("\n")
for word in b:
    word.replace("\n", "")
    word.replace("\n", "")
    word.replace("\n", "")
#create a list
c = []
#now only add stuff that isn't an empty char or does not contain Not Assigned
for word in b:
    if "Not assigned" not in word and word != "":
        c.append(word)
#create a new table to store results
table_contents=[]

for word in c:
    #dict to store values for each neighborhood
    cell = {}
    #Since each postal code is 3 digits, go till the third digit and set it as the postal code value
    cell['Postal Code'] = word[:3]
    #set a temp variable equal to the rest of the word, which has the borough and the neighborhood
    temp = word.replace(word[:3], '')
    #split by the parenthesis, and grab the first half which is the borough, set it to the borough of the cell
    cell['Borough'] = temp.split('(')[0]
    #same logic for the neighborhood, just strip all the uneeded characters
    cell['Neighborhood'] = (((((temp).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
    #append the dict to the table
    table_contents.append(cell)
#convert to trawneigh
trawneigh=pd.DataFrame(table_contents)
#handle special cases
trawneigh['Borough']=trawneigh['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

f = open("/Users/shreyasiyengar/VSC/Coursera_Capstone/Geospatial_Coordinates.csv", "r")
trawnocoords = pd.read_csv("/Users/shreyasiyengar/VSC/Coursera_Capstone/Geospatial_Coordinates.csv",",",header = 0)



trawneigh.head()
    
trawneigh = pd.merge(trawneigh, trawnocoords[['Postal Code', 'Latitude', 'Longitude']], )
trawneigh.head()
    

/Users/shreyasiyengar/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


### Now that we have the neighborhood tables for NYC and Toronto, we need to grab the foursquare venue tables for each city.

#### Foursquare Credential Prep

In [12]:

CLIENT_ID = 'KOGTV4EAQCBU054ZKLCRMAINCATFAHUJWLHPTAUZ4A3N33SG' # your Foursquare ID
CLIENT_SECRET = '4UAMZ5NN21XIQHRK0MRSJ20TYYMF5MABJLQAQEDLRLEPTWGA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: KOGTV4EAQCBU054ZKLCRMAINCATFAHUJWLHPTAUZ4A3N33SG
CLIENT_SECRET:4UAMZ5NN21XIQHRK0MRSJ20TYYMF5MABJLQAQEDLRLEPTWGA


#### The following code, following the New York Lab, gets the nearby venues for a neighborhood using the lat-long, we then use this function to grab the nearby venues for each neighborhood in our dataset.

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

    

In [14]:
trawno_venues = getNearbyVenues(names=trawneigh['Neighborhood'],
                                   latitudes= trawneigh['Latitude'],
                                   longitudes=trawneigh['Longitude']
                                  )
nyc_venues = getNearbyVenues(names=nyneigh['Neighborhood'],
                                   latitudes= nyneigh['Latitude'],
                                   longitudes=nyneigh['Longitude']
                                  )                                 

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [16]:
trawno_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [17]:
nyc_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [30]:
trawveg = trawno_venues.loc[trawno_venues['Venue Category'] == 'Vegetarian / Vegan Restaurant']
trawveg.shape

(17, 7)

In [31]:
nyveg = nyc_venues.loc[nyc_venues['Venue Category'] == 'Vegetarian / Vegan Restaurant']
nyveg.shape

(39, 7)